In [52]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, MaxPooling2D, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [35]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, MaxPooling2D, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.losses import binary_crossentropy

In [15]:
# import zipfile
# zip_ref = zipfile.ZipFile("/content/Face-Mask-Detection-master.zip", 'r')
# zip_ref.extractall("/tmp")
# zip_ref.close()

In [5]:
import os, cv2

In [16]:
data = []
labels = []
for paths, dir, files in os.walk("/content/gdrive/MyDrive/FaceMaskDataset"):
  print(paths)
  for file in files:
    img = os.path.join(paths, file)
    img = load_img(img, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    data.append(img)
    labels.append(paths.split("/")[-1])

/content/gdrive/MyDrive/FaceMaskDataset
/content/gdrive/MyDrive/FaceMaskDataset/with_mask


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


/content/gdrive/MyDrive/FaceMaskDataset/without_mask


In [17]:
lb = LabelEncoder()
labels = lb.fit_transform(labels)

In [18]:
labels

array([0, 0, 0, ..., 1, 1, 1])

In [19]:
labels = to_categorical(labels)

In [20]:
data = np.array(data)
labels = np.array(labels)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [22]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, 
                   horizontal_flip=True, fill_mode="nearest")

In [23]:
basemodel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

9420800/9406464 [==============================] - 0s 0us/step


In [24]:
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size=(7, 7))(headmodel)
headmodel = Flatten(name="Flatten")(headmodel)
headmodel = Dense(128, activation="relu")(headmodel)
headmodel = Dropout(0.5)(headmodel)
headmodel = Dense(2, activation="softmax")(headmodel)

In [25]:
model = Model(inputs = basemodel.input, outputs = headmodel)

In [30]:
# basemodel.layers[0].trainable = False

In [31]:
for layer in basemodel.layers:
  layer.trainable = False

In [40]:
model.compile(loss=binary_crossentropy, optimizer=Adam(lr = 0.0001, decay = 0.0001/20), metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [43]:
new = model.fit(aug.flow(X_train, y_train, batch_size=32), steps_per_epoch=len(X_train)//32, validation_data=(X_test, y_test), validation_steps=len(X_test)//32,
          epochs=20)

Epoch 1/20
95/95 [==============================] - 51s 535ms/step - loss: 0.2547 - accuracy: 0.9450 - val_loss: 0.1164 - val_accuracy: 0.9791
Epoch 2/20
95/95 [==============================] - 39s 410ms/step - loss: 0.1317 - accuracy: 0.9651 - val_loss: 0.0705 - val_accuracy: 0.9844
Epoch 3/20
95/95 [==============================] - 38s 404ms/step - loss: 0.0917 - accuracy: 0.9776 - val_loss: 0.0546 - val_accuracy: 0.9844
Epoch 4/20
95/95 [==============================] - 38s 399ms/step - loss: 0.0733 - accuracy: 0.9806 - val_loss: 0.0521 - val_accuracy: 0.9804
Epoch 5/20
95/95 [==============================] - 40s 417ms/step - loss: 0.0640 - accuracy: 0.9835 - val_loss: 0.0421 - val_accuracy: 0.9844
Epoch 6/20
95/95 [==============================] - 39s 411ms/step - loss: 0.0601 - accuracy: 0.9825 - val_loss: 0.0440 - val_accuracy: 0.9817
Epoch 7/20
95/95 [==============================] - 39s 411ms/step - loss: 0.0545 - accuracy: 0.9822 - val_loss: 0.0405 - val_accuracy: 0.9817

In [45]:
pred = model.predict(X_test, batch_size=32)

In [46]:
np.argmax(pred, axis=1)

array([0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,

In [47]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [58]:
model.save("Mask_detector.model", save_format="h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [60]:
os.getcwd()

'/content'